In [1]:
import boto3
import pandas as pd
from datetime import datetime, timedelta
import os
import io

In [14]:
service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
access_key = 'cPKhn3d9z9ZWYMAPPwyq'
secret_key = 'VFjEkpateO6VFGY0U9op073I4NXTlejMBMIq6FvA'
bucket_name = 'bucket1chaegpt'

### 파일 가져오기

In [15]:
usa_list = pd.read_csv('해외_산업분류(최종).csv')['Ticker'].unique()
usa_list

array(['AA', 'ACNT', 'ALB', ..., 'MOG.A', 'UHAL.B', 'CWEN.A'],
      dtype=object)

In [16]:
#클라우드에서 주가데이터 불러오기
def download_stock_data(bucket_name, ticker, local_file_path):
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)
    
    file_name = f'{ticker}_주가데이터.xlsx'
    object_name = f'usa_stock_crawling/{file_name}'

    s3.download_file(bucket_name, object_name, local_file_path)

In [17]:
# 6개월로 기간 줄이기
def six_month(df):
    df['Date'] = pd.to_datetime(df['Date'])
    
    recent_date = df['Date'].max()
    
    start_date = recent_date - pd.DateOffset(months = 6)
    start_date = start_date.replace(microsecond = 0, nanosecond = 0)
    
    df = df[(df['Date'] >= start_date) & (df['Date'] <= recent_date)]
    
    return df

In [18]:
#변동이 큰 구간 4개 추출하기(start_date, end_date, duration, change_average 계산)
def process_dataframe(df):
    selected_rows = []
    consecutive_count_up = 0
    consecutive_count_down = 0
    start_date_up = None
    start_date_down = None
    change_sum_up = 0
    change_sum_down = 0

    for index, row in df.iterrows():
        if row['Change'] >= 0:  # 양수에서 음수로 바뀐 경우
            if consecutive_count_up == 0:
                start_date_up = row['Date']  # 양수에서 음수로 바뀐 날짜 기록
            consecutive_count_up += 1
            change_sum_up += row['Change']
        elif row['Change'] <= 0:  # 음수에서 양수로 바뀐 경우
            if consecutive_count_up > 0:
                # 이전 row의 값을 사용하여 'End Date' 설정
                prev_row = df.loc[index - 1]
                selected_rows.append({
                    'Start Date': start_date_up,
                    'End Date': prev_row['Date'],
                    'Duration': consecutive_count_up,  # 바뀐 이후 다시 양수로 바뀌는 시점까지의 기간
                    'Change Average': change_sum_up / (consecutive_count_up)  # Duration 동안의 Change 평균
                })
            consecutive_count_up = 0
            change_sum_up = 0

        if row['Change'] <= 0:  # 음수에서 양수로 바뀐 경우
            if consecutive_count_down == 0:
                start_date_down = row['Date']  # 음수에서 양수로 바뀐 날짜 기록
            consecutive_count_down += 1
            change_sum_down += row['Change']
        elif row['Change'] >= 0:  # 양수에서 음수로 바뀐 경우
            if consecutive_count_down > 0:
                # 이전 row의 값을 사용하여 'End Date' 설정
                prev_row = df.loc[index - 1]
                selected_rows.append({
                    'Start Date': start_date_down,
                    'End Date': prev_row['Date'],
                    'Duration': consecutive_count_down,  # 바뀐 이후 다시 음수로 바뀌는 시점까지의 기간
                    'Change Average': change_sum_down / (consecutive_count_down)  # Duration 동안의 Change 평균
                })
            consecutive_count_down = 0
            change_sum_down = 0

    if selected_rows:
        selected_df = pd.DataFrame(selected_rows)
        selected_df = selected_df[selected_df['Duration'] > 1]
        s_df = selected_df.copy()
        sorted_index = s_df['Change Average'].abs().sort_values(ascending = False).index
        sorted_selected_df = selected_df.loc[sorted_index].head(4)
        return sorted_selected_df
    else:
        return pd.DataFrame()  # 빈 데이터프레임 반환

In [19]:
#클라우드에 업로드
def upload(ticker, data, s3_client, bucket_name, endpoint_url, access_key, secret_key):

    # 데이터프레임을 파일로 저장
    xlsx_filename = f'{ticker}_차트분석.xlsx'
    xlsx_buffer = io.BytesIO()
    data.to_excel(xlsx_buffer, index = False)
    xlsx_buffer.seek(0)

    # xlsx 파일 클라우드 업로드
    object_name = f'stock_chart_usa/{xlsx_filename}'
    s3_client.upload_fileobj(xlsx_buffer, bucket_name, object_name)
    print(f"Uploaded {xlsx_filename} to S3")

In [20]:
for tick in usa_list:
    file_path = f'C:/Users/pc/미래에셋/미국_주가데이터/{tick}_주가데이터.xlsx'
    
    # 데이터 가져오기
    download_stock_data(bucket_name, tick, file_path)
    
    # 데이터프레임으로 읽어오기
    st_data = pd.read_excel(file_path)
    
    result_df = process_dataframe(st_data)
    result_df['Ticker'] = tick
    
    # 파일 삭제
    if os.path.exists(file_path):
        os.remove(file_path)
    else :
        print(f"파일이 존재하지 않습니다: {file_path}")
        
    # AWS S3 클라이언트 생성
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    # 클라우드 폴더 생성
    object_name = 'stock_chart_usa/'
    s3.put_object(Bucket=bucket_name, Key=object_name)

    # 주가 데이터 수집 및 파일로 저장
    upload(tick, result_df, s3, bucket_name, endpoint_url, access_key, secret_key)